# Modele analizy danych

Tomasz Rodak

---

### Zadanie 12.1

Rozwiąż zadania 5.4.1 i 5.4.2 z ISLP.

### Zadanie 12.2

Rozwiąż zadanie 5.4.6 z ISLP.

In [5]:
!pip install ISLP

In [6]:
from ISLP import load_data
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [7]:
df = load_data('Default')
df.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [8]:
#Usuwamy student bo nie bedziem uzywac
df = df.drop(columns=['student'])
df.head()

,default,balance,income
0,No,729.526495,44361.625074
1,No,817.180407,12106.134700
2,No,1073.549164,31767.138947
3,No,529.250605,35704.493935
4,No,785.655883,38463.495879


In [9]:
df["default"] = pd.get_dummies(df['default'], drop_first=True, dtype=int, prefix='default')
df.head()

,default,balance,income
0,0,729.526495,44361.625074
1,0,817.180407,12106.134700
2,0,1073.549164,31767.138947
3,0,529.250605,35704.493935
4,0,785.655883,38463.495879


In [10]:
sm.GLM(df['default'], df[['balance', 'income']], family=sm.families.Binomial()).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1734.6
Date:                Mon, 12 Jan 2026   Deviance:                       3469.1
Time:                        18:15:14   Pearson chi2:                 3.87e+04
No. Iterations:                     7   Pseudo R-squ. (CS):           -0.05638
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
balance        0.0004   7.03e-05      5.797      0.000       0.000       0.001
income        -0.0001    3.7e-06    -34.025      0.000      -0.000      -0.000
==============================================================================
"""

In [11]:
df.describe()

,default,balance,income
count,10000.000000,10000.000000,10000.000000
mean,0.033300,835.374886,33516.981876
std,0.179428,483.714985,13336.639563
min,0.000000,0.000000,771.967729
25%,0.000000,481.731105,21340.462903
50%,0.000000,823.636973,34552.644802
75%,0.000000,1166.308386,43807.729272
max,1.000000,2654.322576,73554.233495


In [12]:
model = sm.GLM(df['default'], sm.add_constant(df[['balance', 'income']]), family=sm.families.Binomial())
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Mon, 12 Jan 2026   Deviance:                       1579.0
Time:                        18:15:14   Pearson chi2:                 6.95e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1256
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================
"""

In [13]:
def boot_SE(func,
           D,
           n=None,
           B=1000,
           seed=None):
  n = len(D) if n is None else n
  rng = np.random.default_rng(seed)
  first_, second_ = 0, 0
  for _ in range(B):
    idx= rng.choice(D.index, size=n, replace=True)
    value = func(D, idx)
    first_ += value
    second_ += value**2
  return np.sqrt(second_ / B - (first_ / B)**2)

In [14]:
def boot_fn(D, idx):
  df_ = D.loc[idx]
  model = sm.GLM.from_formula('default ~ balance + income', data=df_, family=sm.families.Binomial())
  model = model.fit()
  return model.params

In [15]:
boot_bse = boot_SE(boot_fn, df)
boot_bse

,0
Intercept,0.432716
balance,0.000226
income,0.000005


In [16]:
model.bse

,0
const,0.434772
balance,0.000227
income,0.000005


In [17]:
np.abs(model.bse - boot_bse) / model.bse

,0
Intercept,NaN
balance,0.006859
const,NaN
income,0.003733


### Zadanie 12.3

Kontynuacja zadania 12.2

Ustal `balance = 1000` i `income = 20_000`. Oblicz przewidywane prawdopodobieństwo `default` oraz logarytm szans (log-odds) dla tych wartości predyktorów.

In [18]:
balance, income = 1000, 20000
p = model.predict([1, balance, income]).item()
p

0.004163670127881266

In [19]:
np.log(p / (1 - p))

np.float64(-5.477185989038384)

Wyznacz 95% przedział ufności dla przewidywanego prawdopodobieństwa `default` wykorzystując metodę `get_prediction([1, balance, income])` obiektu `model`.

In [20]:
pred = model.get_prediction([1, balance, income])
pred.summary_frame(alpha=0.05)

,mean,mean_se,mean_ci_lower,mean_ci_upper
0,0.004164,0.000732,0.00295,0.005874


Wyznacz 95% przedział ufności dla przewidywanego logarytmu szans (log-odds) metodą bootstrapu wykorzystując:
* odchylenie standardowe oszacowane z powtórzeń bootstrapowych,
* kwantyle rozkładu bootstrapowego.

In [22]:
import numpy as np

B = 1000
boot_log_odds = np.zeros(B)
rng = np.random.default_rng()
for i in range(B):
  idx = rng.choice(df.index, size=len(df), replace=True)
  df_ = df.loc[idx]
  model = sm.GLM(df_['default'], sm.add_constant(df_[['balance', 'income']]), family=sm.families.Binomial())
  model = model.fit()
  p = model.predict([1, balance, income]).item()
  boot_log_odds[i] = np.log(p / (1 - p))

In [23]:
boot_log_odds.shape

(1000,)

In [24]:
std_boot = boot_log_odds.std()
std_boot

np.float64(0.1771489883759188)

In [25]:
log_odd = np.log(p / (1 - p)) #logarytm szansy dla ustalonych balande i income
log_odd

np.float64(-5.56720718625066)

In [26]:
log_odd - 1.96 * std_boot, log_odd + 1.96 * std_boot

(np.float64(-5.914419203467461), np.float64(-5.219995169033859))

In [27]:
np.quantile(boot_log_odds, [0.025, 0.975]) #kwantylowy CI dla log_odd

array([-5.84596955, -5.16665114])

Wykonaj transformację sigmoidową, aby uzyskać przedziały ufności dla przewidywanego prawdopodobieństwa `default` na podstawie przedziałów dla log-odds.

Porównaj oba przedziały ufności z wynikiem z metody `get_prediction`.

In [28]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [29]:
sigmoid(log_odd - 1.96 * std_boot), sigmoid(log_odd + 1.96 * std_boot) # CI dla p

(np.float64(0.0026929560320820695), np.float64(0.005378273016302037))

In [30]:
sigmoid(np.quantile(boot_log_odds, [0.025, 0.975])) #kwantylowy CI dla p

array([0.00288319, 0.00567129])